In [250]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [251]:
df = pd.read_csv('../Dataset/Dataset_EditActions.csv')
df = df.drop(['Architecture', 'Reproducible?', 'Code Snippet Present?', 'System Configuration Present?', 'Data Description Present?', 'Framework'], axis = 1)

In [252]:
# Replace Training Bug with value T, Model bug with value M, API bug with value A, Tensor & Input Bug with value I, and GPU Bug with value G
df.loc[df['Type of Bug'] == 'Training Bug', 'Type of Bug'] = 'T'
df.loc[df['Type of Bug'] == 'Model Bug', 'Type of Bug'] = 'M'
df.loc[df['Type of Bug'] == 'API Bug', 'Type of Bug'] = 'A'
df.loc[df['Type of Bug'] == 'Tensor and Input Bug', 'Type of Bug'] = 'I'
df.loc[df['Type of Bug'] == 'GPU Bug', 'Type of Bug'] = 'G'
df.loc[df['Type of Bug'] == 'Mixed Bug', 'Type of Bug'] = 'X'

In [253]:
# Map values from new columns as the following:
# Input Data Generation: D 
# Neural Network Definition: N
# Obsolete Parameter Removal: O
# Framework Migration: F
# Dataset Procurement: P
# Models and Tokenizers: M
# Logging: L
# Import Addition and Dependency Resolution: R
# Compiler Error Resolution: C
# Hyperparameter Initialization: H

# Replace all 1's with the corresponding letter
df['Input Data Generation'] = df['Input Data Generation'].replace(1, 'D')
df['Neural Network Definition'] = df['Neural Network Definition'].replace(1, 'N')
df['Obsolete Parameter Removal'] = df['Obsolete Parameter Removal'].replace(1, 'O')
df['Framework Migration'] = df['Framework Migration'].replace(1, 'F')
df['Dataset Procurement'] = df['Dataset Procurement'].replace(1, 'P')
df['Downloading Models and Tokenizers'] = df['Downloading Models and Tokenizers'].replace(1, 'M')
df['Logging'] = df['Logging'].replace(1, 'L')
df['Import Addition and Dependency Resolution'] = df['Import Addition and Dependency Resolution'].replace(1, 'R')
df['Compiler Error Resolution'] = df['Compiler Error Resolution'].replace(1, 'C')
df['Hyperparameter Initialization'] = df['Hyperparameter Initialization'].replace(1, 'H')


df['Input Data Generation'] = df['Input Data Generation'].replace(0, '')
df['Neural Network Definition'] = df['Neural Network Definition'].replace(0, '')
df['Obsolete Parameter Removal'] = df['Obsolete Parameter Removal'].replace(0, '')
df['Framework Migration'] = df['Framework Migration'].replace(0, '')
df['Dataset Procurement'] = df['Dataset Procurement'].replace(0, '')
df['Downloading Models and Tokenizers'] = df['Downloading Models and Tokenizers'].replace(0, '')
df['Logging'] = df['Logging'].replace(0, '')
df['Import Addition and Dependency Resolution'] = df['Import Addition and Dependency Resolution'].replace(0, '')
df['Compiler Error Resolution'] = df['Compiler Error Resolution'].replace(0, '')
df['Hyperparameter Initialization'] = df['Hyperparameter Initialization'].replace(0, '')

In [281]:
training_bugs = df[df['Type of Bug'] == 'T']
gpu_bugs = df[df['Type of Bug'] == 'G']
api_bugs = df[df['Type of Bug'] == 'A']
model_bugs = df[df['Type of Bug'] == 'M']
tensor_bugs = df[df['Type of Bug'] == 'I']
mixed_bugs = df[df['Type of Bug'] == 'X']

training_transactions = []
for i in range(0, len(training_bugs)):
    training_transactions.append([str(training_bugs.values[i,j]) for j in range(0, len(training_bugs.columns)) if str(training_bugs.values[i,j]) != ''])

gpu_transactions = []
for i in range(0, len(gpu_bugs)):
    gpu_transactions.append([str(gpu_bugs.values[i,j]) for j in range(0, len(gpu_bugs.columns)) if str(gpu_bugs.values[i,j]) != ''])

api_transactions = []
for i in range(0, len(api_bugs)):
    api_transactions.append([str(api_bugs.values[i,j]) for j in range(0, len(api_bugs.columns)) if str(api_bugs.values[i,j]) != ''])

model_transactions = []
for i in range(0, len(model_bugs)):
    model_transactions.append([str(model_bugs.values[i,j]) for j in range(0, len(model_bugs.columns)) if str(model_bugs.values[i,j]) != ''])

tensor_transactions = []
for i in range(0, len(tensor_bugs)):
    tensor_transactions.append([str(tensor_bugs.values[i,j]) for j in range(0, len(tensor_bugs.columns)) if str(tensor_bugs.values[i,j]) != ''])

mixed_transactions = []
for i in range(0, len(mixed_bugs)):
    mixed_transactions.append([str(mixed_bugs.values[i,j]) for j in range(0, len(mixed_bugs.columns)) if str(mixed_bugs.values[i,j]) != ''])

In [297]:
def get_apriori_results(rules, character):
    for result in list(rules):
     if character in result.items:
        items = ", ".join(result.items)
        support = result.support
        print(f"Items: {{{items}}}")
        print(f"Support: {support:.4f}")
        if result.ordered_statistics:
            print("Association Rules:")
            for rule in result.ordered_statistics:
                if character in rule.items_base:
                    antecedent = ", ".join(rule.items_base)
                    consequent = ", ".join(rule.items_add)
                    confidence = rule.confidence
                    lift = rule.lift
                    print(f"  {{{antecedent}}} => {{{consequent}}}")
                    print(f"Confidence: {confidence:.4f}")

In [298]:
get_apriori_results(apriori(training_transactions, max_length = 2), 'T')

Items: {T}
Support: 1.0000
Association Rules:
Items: {C, T}
Support: 0.2692
Association Rules:
  {T} => {C}
Confidence: 0.2692
Items: {T, D}
Support: 0.6154
Association Rules:
  {T} => {D}
Confidence: 0.6154
Items: {F, T}
Support: 0.3462
Association Rules:
  {T} => {F}
Confidence: 0.3462
Items: {T, H}
Support: 0.3077
Association Rules:
  {T} => {H}
Confidence: 0.3077
Items: {T, L}
Support: 0.2692
Association Rules:
  {T} => {L}
Confidence: 0.2692
Items: {T, N}
Support: 0.1154
Association Rules:
  {T} => {N}
Confidence: 0.1154
Items: {T, O}
Support: 0.1538
Association Rules:
  {T} => {O}
Confidence: 0.1538
Items: {P, T}
Support: 0.2692
Association Rules:
  {T} => {P}
Confidence: 0.2692
Items: {R, T}
Support: 0.4231
Association Rules:
  {T} => {R}
Confidence: 0.4231


In [299]:
get_apriori_results(apriori(model_transactions, max_length = 2), 'M')

Items: {M}
Support: 1.0000
Association Rules:
Items: {C, M}
Support: 0.3000
Association Rules:
  {M} => {C}
Confidence: 0.3000
Items: {M, D}
Support: 0.3500
Association Rules:
  {M} => {D}
Confidence: 0.3500
Items: {F, M}
Support: 0.2000
Association Rules:
  {M} => {F}
Confidence: 0.2000
Items: {M, H}
Support: 0.5000
Association Rules:
  {M} => {H}
Confidence: 0.5000
Items: {L, M}
Support: 0.4500
Association Rules:
  {M} => {L}
Confidence: 0.4500
Items: {N, M}
Support: 0.7500
Association Rules:
  {M} => {N}
Confidence: 0.7500
Items: {M, O}
Support: 0.1500
Association Rules:
  {M} => {O}
Confidence: 0.1500
Items: {P, M}
Support: 0.4000
Association Rules:
  {M} => {P}
Confidence: 0.4000
Items: {R, M}
Support: 0.5500
Association Rules:
  {M} => {R}
Confidence: 0.5500


In [300]:
get_apriori_results(apriori(gpu_transactions, max_length = 2), 'G')

Items: {G}
Support: 1.0000
Association Rules:


In [301]:
get_apriori_results(apriori(tensor_transactions, max_length = 2), 'I')

Items: {I}
Support: 1.0000
Association Rules:
Items: {C, I}
Support: 0.3333
Association Rules:
  {I} => {C}
Confidence: 0.3333
Items: {D, I}
Support: 0.5714
Association Rules:
  {I} => {D}
Confidence: 0.5714
Items: {I, H}
Support: 0.5238
Association Rules:
  {I} => {H}
Confidence: 0.5238
Items: {L, I}
Support: 0.4286
Association Rules:
  {I} => {L}
Confidence: 0.4286
Items: {I, O}
Support: 0.3333
Association Rules:
  {I} => {O}
Confidence: 0.3333
Items: {P, I}
Support: 0.1905
Association Rules:
  {I} => {P}
Confidence: 0.1905
Items: {R, I}
Support: 0.4762
Association Rules:
  {I} => {R}
Confidence: 0.4762


In [302]:
get_apriori_results(apriori(mixed_transactions, max_length = 2), 'X')

Items: {X}
Support: 1.0000
Association Rules:
Items: {C, X}
Support: 0.5000
Association Rules:
  {X} => {C}
Confidence: 0.5000
Items: {X, D}
Support: 0.7500
Association Rules:
  {X} => {D}
Confidence: 0.7500
Items: {F, X}
Support: 0.2500
Association Rules:
  {X} => {F}
Confidence: 0.2500
Items: {X, H}
Support: 1.0000
Association Rules:
  {X} => {H}
Confidence: 1.0000
Items: {N, X}
Support: 0.2500
Association Rules:
  {X} => {N}
Confidence: 0.2500
Items: {P, X}
Support: 0.2500
Association Rules:
  {X} => {P}
Confidence: 0.2500
Items: {R, X}
Support: 0.7500
Association Rules:
  {X} => {R}
Confidence: 0.7500
